In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [3]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

scaler.fit(df_market_data)

scaled_data = scaler.transform(df_market_data)

In [7]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data, columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index
# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [8]:
# Create a list with the number of k-values from 2 to 20
k = list(range(2, 21))

In [9]:
# Create an empty list to store the inertia values
inertia = []
silhouettes = []
cha_scores = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:

    # Initialize the K-Means model
    k_model = KMeans(n_clusters=i, random_state=1)

    # Fit the K-Means model
    k_model.fit(df_market_data_scaled)

    # Make predictions using the K-Means model
    preds = k_model.predict(df_market_data_scaled)

    # Append the inerties to the inertia list
    inertia.append(k_model.inertia_)

    # Append the silhouette score to the silhouettes list
    silhouettes.append(silhouette_score(df_market_data_scaled, preds))

    # Append the Calinski-Harabasz score to the cha_scores list
    cha_scores.append(calinski_harabasz_score(df_market_data_scaled, preds))


In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia, "silhouette": silhouettes, "calinski_harabasz": cha_scores}

# Create a DataFrame with the data to plot the Elbow curve
scaled_elbow_df = pd.DataFrame(elbow_data)
scaled_elbow_df["acc"] = scaled_elbow_df.inertia.diff()

scaled_elbow_df

,k,inertia,silhouette,calinski_harabasz,acc
0,2,212.123342,0.690885,13.766470,NaN
1,3,165.136752,0.260810,14.021117,-46.986590
2,4,79.022435,0.314482,32.459853,-86.114317
3,5,66.413051,0.323913,29.892958,-12.609385
4,6,61.634088,0.251616,25.595599,-4.778963
5,7,46.074467,0.258868,29.631263,-15.559621
6,8,43.392694,0.252185,26.466078,-2.681773
7,9,32.913223,0.260185,30.879599,-10.479471
8,10,30.469111,0.168099,29.000071,-2.444111
9,11,26.480101,0.175935,29.514981,-3.989010


In [11]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
scaled_elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Probably 4 based on the Elbow Curve, the Silhouette score, and the Calinski-Harabasz score.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [12]:
# Initialize the K-Means model using the best value for k
scaled_model = KMeans(n_clusters=4, random_state=1)

In [13]:
# Fit the K-Means model using the scaled data
scaled_model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled data
preds = scaled_model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(preds)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [15]:
# Create a copy of the DataFrame
scaled_preds_df = df_market_data_scaled.copy()

In [16]:
# Add a new column to the DataFrame with the predicted clusters
scaled_preds_df["clusters"] = preds

# Display sample data
scaled_preds_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [17]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_preds_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"]
)

:NdOverlay   [clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [19]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_market_data_scaled)
pca_df = pd.DataFrame(data=pca_data, columns=["PC_1", "PC_2", "PC_3"])
# View the first five rows of the DataFrame. 
pca_df.head()

,PC_1,PC_2,PC_3
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


In [20]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance_ratio = pca.explained_variance_ratio_
print(explained_variance_ratio)
print(sum(explained_variance_ratio))

[0.3719856  0.34700813 0.17603793]
0.8950316570309842


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%. This is good enough for this application to justify using 3 principal components.

In [21]:
# Create a new DataFrame with the PCA data.

# Copy the crypto names from the original data
pca_df["coin_id"] = df_market_data.index

# Set the coinid column as index
pca_df = pca_df.set_index("coin_id")

# For plotting comparisons
pca_with_scaled_df = pca_df.merge(df_market_data_scaled, how="left", left_on="coin_id", right_index=True)

# Display sample data
pca_df.head()

,PC_1,PC_2,PC_3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [22]:
# Create a list with the number of k-values from 1 to 11
# can use same k values as in the elbow curve above

In [23]:
# Create an empty list to store the inertia values
inertia = []
silhouettes = []
cha_scores = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:

    # Initialize the K-Means model
    k_model = KMeans(n_clusters=i, random_state=1)

    # Fit the K-Means model
    k_model.fit(pca_df)

    # Make predictions using the K-Means model
    preds = k_model.predict(pca_df)

    # Append the inerties to the inertia list
    inertia.append(k_model.inertia_)

    # Append the silhouette score to the silhouettes list
    silhouettes.append(silhouette_score(pca_df, preds))

    # Append the Calinski-Harabasz score to the cha_scores list
    cha_scores.append(calinski_harabasz_score(pca_df, preds))

In [24]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia, "silhouette": silhouettes, "cha": cha_scores}
# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(elbow_data)
pca_elbow_df["acc"] = pca_elbow_df.inertia.diff()

pca_elbow_df

,k,inertia,silhouette,cha,acc
0,2,182.339530,0.728406,15.941950,NaN
1,3,135.442408,0.356130,17.034560,-46.897122
2,4,49.665497,0.420800,51.455694,-85.776911
3,5,38.672582,0.405662,50.780512,-10.992914
4,6,34.507577,0.334511,45.107936,-4.165006
5,7,23.728365,0.375378,55.678472,-10.779212
6,8,18.574523,0.316385,60.481348,-5.153841
7,9,15.572672,0.323751,61.980734,-3.001851
8,10,11.407667,0.350621,74.116423,-4.165006
9,11,8.058822,0.361557,92.624679,-3.348845


In [25]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `k` is 4. This is based on the Elbow Curve, the Silhouette score, and the Calinski-Harabasz score.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ from the best k value found using the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [26]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4, random_state=1)

In [27]:
# Fit the K-Means model using the PCA data
pca_model.fit(pca_df)

KMeans(n_clusters=4, random_state=1)

In [28]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_preds = pca_model.predict(pca_df)
# Print the resulting array of cluster values.
print(pca_preds)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [29]:
# Create a copy of the DataFrame with the PCA data
pca_preds_df = pca_with_scaled_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_preds_df["clusters"] = pca_preds

# Display sample data
pca_preds_df.head()

,PC_1,PC_2,PC_3,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,clusters
coin_id,,,,,,,,,,,
bitcoin,-0.600667,0.842760,0.461595,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,-0.458261,0.458466,0.952877,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,-0.433070,-0.168126,-0.641752,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.471835,-0.222660,-0.479053,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,-1.157800,2.041209,1.859715,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [30]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_preds_df.hvplot.scatter(
    x="PC_1",
    y="PC_2",
    by="clusters",
    hover_cols=["coin_id"]
)

:NdOverlay   [clusters]
   :Scatter   [PC_1]   (PC_2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [31]:
# Composite plot to contrast the Elbow curves
scaled_elbow_curve = scaled_elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k, label="scaled")
pca_elbow_curve = pca_elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k, label="pca")
scaled_elbow_curve * pca_elbow_curve

:Overlay
   .Curve.Scaled :Curve   [k]   (inertia)
   .Curve.Pca    :Curve   [k]   (inertia)

In [36]:
# Composite plot to contrast the clusters
pca_scatter = pca_preds_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
    label = "pca",
    width=350
)

scaled_scatter = scaled_preds_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
    label = "scaled",
    width=350
)

(pca_scatter + scaled_scatter).opts(title="PCA vs. Scaled")

:Layout
   .NdOverlay.Pca    :NdOverlay   [clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.Scaled :NdOverlay   [clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** There seems to be no difference in the impact of using fewer features to cluster the data using K-Means. The only difference is in the elbow curve. Future analysis should be focused on using different modeling techniques like Agglomerative Clustering and Random Forest on a larger sample of data.